In [1]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tflearn

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tensorflow==1.14

Note: you may need to restart the kernel to use updated packages.


In [5]:
#making important imports and loading the json data
import nltk 
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random

import json 
with open('intents.json') as file:
    data=json.load(file)

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [6]:
#extracting data
words=[]
labels=[]
docs_x=[]
docs_y=[]

In [7]:
#turning into a list of words 
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds=nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [8]:
#Stemming of words to reduce the vocabulary of the model
words=[stemmer.stem(w.lower()) for w in words if w!= "?"]
words=sorted(list(set(words)))

labels=sorted(labels)

In [9]:
#creating a bag of words
training=[]
output=[]

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag=[]
    
    wrds=[stemmer.stem(w.lower()) for w in doc]
    
    for w in words:
        if w in words:
            bag.append(1)
        else:
            bag.append(0)
            
    output_row=out_empty[:]
    output_row[labels.index(docs_y[x])]=1
    
    training.append(bag)
    output.append(output_row)

In [10]:
#finally we will convert our training data and output to numpy arrays
training = numpy.array(training)
output=numpy.array(output)

In [11]:
#developing the model
tensorflow.reset_default_graph()
net=tflearn.input_data(shape=[None,len(training[0])])
net=tflearn.fully_connected(net,8)
net=tflearn.fully_connected(net,8)
net=tflearn.fully_connected(net,len(output[0]),activation="softmax")
net=tflearn.regression(net)

model=tflearn.DNN(net)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [12]:
#training and saving the model
model.fit(training, output, n_epoch=1000,batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 3999  | total loss: 1.76059 | time: 0.012s
| Adam | epoch: 1000 | loss: 1.76059 - acc: 0.2550 -- iter: 24/26
Training Step: 4000  | total loss: 1.77056 | time: 0.014s
| Adam | epoch: 1000 | loss: 1.77056 - acc: 0.2295 -- iter: 26/26
--
INFO:tensorflow:C:\Users\MEGHAJ SINGH\Desktop\simple_chat_bot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
#making predictions
def bag_of_words(s,words):
    bag=[0 for _ in range(len(words))]
    
    s_words=nltk.word_tokenize(s)
    s_words=[stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i,w in enumerate(words):
            if w==se:
                bag[i]=1
    return numpy.array(bag)

def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp=input("You: ")
        if inp.lower()=="quit":
            break
            
        results=model.predict([bag_of_words(inp,words)])
        results_index=numpy.argmax(results)
        tag=labels[results_index]
        
        for tg in data["intents"]:
            if tg['tag']==tag:
                responses=tg['responses']
                
        print(random.choice(responses))
        
chat()

Start talking with the bot (type quit to stop)!
You: Hi
Good to see you again!
You: old?
Good to see you again!
You: why?
Hello!
You: what?
Hi there, how can I help?
You: i love it
Hello!
